In [1]:
from torch import nn
from torch import optim
from model import TextRNN
from cnews_loader import get_labels,get_vocab,process_file
import torch.utils.data as Data
import numpy as np
import torch

train_file = 'cnews.train.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

def train():
    model = TextRNN().cuda()#设置使用GPU
    Loss = nn.CrossEntropyLoss()#定义损失函数(此标准将LogSoftMax和NLLLoss集成到一个类中。
                                            #当训练一个多类分类器的时候，这个方法是十分有用的。)                                            
    optimer = optim.Adam(model.parameters(), lr=0.001)#model.parameters():待优化参数的iterable或者是定义了参数组的dict
    #当网络的评价指标不在提升的时候，可以通过降低网络的学习率来提高网络性能                                                  #lr=0.001:学习率
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimer,#optimer指的是网络的优化器
                                                           mode='max', #mode (str) ，可选择‘min’或者‘max’，min表示当监控量停止下降的时候，学习率将减小，max表示当监控量停止上升的时候，学习率将减小。默认值为‘min’
                                                           patience=5,#容忍网路的性能不提升的次数，高于这个次数就降低学习率
                                                           verbose=True,#如果为True，则为每次更新向stdout输出一条消息。 默认值：False
                                                           min_lr=1.e-6)# 学习率下限
    best_val_acc = 0
    
    
    total_iter=0
    for epoch in range(30):
        
        model.train()
        for step,(batch_x, batch_y) in enumerate(train_loader):
            
            
            
            total_iter+=1
            x = batch_x.cuda()
            y = batch_y.cuda().long()
            y_label= torch.argmax(y,1)
            
            out = model(x)
            loss = Loss(out, y_label)
            optimer.zero_grad()#梯度清零
            loss.backward()#反向传播
            optimer.step()
            
            preds=torch.softmax(out,dim=1)
            acc_train = np.mean((torch.argmax(preds,1) == torch.argmax(y,1)).cpu().numpy())
            
            if step%100==0:
               
                print("epoch:{},step:{},acc_train:{},loss_train:{}".format(epoch,total_iter,acc_train,loss))
            
        acc_val_val=0
        
        
        model.eval()
        for step,(batch_x, batch_y) in enumerate(val_loader):
            
            x = batch_x.cuda()
            y = batch_y.cuda().long()
            y_label= torch.argmax(y,1)
            out = model(x)
            
            preds=torch.softmax(out,dim=1)
            acc_val = np.mean((torch.argmax(preds,1) == torch.argmax(y,1)).cpu().numpy())
            acc_val_val+=acc_val
            if acc_val > best_val_acc:
                torch.save(model.state_dict(), './model_params.pkl')
                best_val_acc = acc_val
        acc_val_=acc_val_val/(step+1)
        print("epoch:{},step:{},acc_val:{}".format(epoch,total_iter,acc_val_))
        scheduler.step(acc_val_)

            
# 获取文本的类别及其对应id的字典
label, label_dict = get_labels()
# 获取训练文本中所有出现过的字及其所对应的id
word, word_dict = get_vocab('cnews.vocab.txt')
# 数据加载及分批
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file('cnews.train.txt', word_dict, label_dict, 600)
x_val, y_val = process_file('cnews.val.txt', word_dict, label_dict, 600)
x_test, y_test = process_file('cnews.test.txt', word_dict, label_dict, 600)

x_train, y_train = torch.LongTensor(x_train),torch.Tensor(y_train)
x_val,y_val = torch.LongTensor(x_val),torch.Tensor(y_val)

train_dataset = Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size=128,
                              shuffle=True,num_workers=2)
val_dataset = Data.TensorDataset(x_val,y_val)
val_loader = Data.DataLoader(dataset=val_dataset,batch_size=128,
                            shuffle=True,num_workers=2)


Using TensorFlow backend.


In [2]:
train()

epoch:0,step:1,acc_train:0.078125,loss_train:2.3147175312042236
epoch:0,step:101,acc_train:0.5546875,loss_train:1.3804938793182373
epoch:0,step:201,acc_train:0.734375,loss_train:0.9010396003723145
epoch:0,step:301,acc_train:0.7890625,loss_train:0.6187540888786316
epoch:0,step:391,acc_val:0.8076171875
epoch:1,step:392,acc_train:0.78125,loss_train:0.5959916710853577
epoch:1,step:492,acc_train:0.8515625,loss_train:0.45020559430122375
epoch:1,step:592,acc_train:0.828125,loss_train:0.5411161184310913
epoch:1,step:692,acc_train:0.8671875,loss_train:0.39501336216926575
epoch:1,step:782,acc_val:0.851171875
epoch:2,step:783,acc_train:0.9140625,loss_train:0.27249738574028015
epoch:2,step:883,acc_train:0.9453125,loss_train:0.1866016834974289
epoch:2,step:983,acc_train:0.9453125,loss_train:0.24887068569660187
epoch:2,step:1083,acc_train:0.8671875,loss_train:0.36486995220184326
epoch:2,step:1173,acc_val:0.875
epoch:3,step:1174,acc_train:0.90625,loss_train:0.44178131222724915
epoch:3,step:1274,acc_t

In [7]:
# 测试
import keras as kr
import torch
from torch import nn
from cnews_loader import get_labels,get_vocab,process_file
from model import TextRNN
import numpy as np
 
vocab_file = 'cnews.vocab.txt'
 
class RnnModel:
    def __init__(self):
        self.label, self.label_dict = get_labels()
        self.words, self.word_dict = get_vocab(vocab_file)
        self.model = TextRNN()
        self.model.load_state_dict(torch.load('model_params.pkl'))
 
    def predict(self, message):
        content = message
        data = [self.word_dict[x] for x in content if x in self.word_dict]
        data = kr.preprocessing.sequence.pad_sequences([data], 600)
        data = torch.LongTensor(data)
        y_pred_cls = self.model(data)
        class_index = torch.argmax(y_pred_cls[0]).item()
        return self.label[class_index]
 
 
if __name__ == '__main__':
    model = RnnModel()
    test_demo = ['《时光重返四十二难》恶搞唐增取经一款时下最热门的动画人物：猪猪侠，加上创新的故事背景，震撼的操作快感，成就了这部恶搞新作，现正恶搞上市，玩家们抢先赶快体验快感吧。游戏简介：被时光隧道传送到208年的猪猪侠，必须经历六七四十二难的考验，才能借助柯伊诺尔大钻石的力量，开启时光隧道，重返2008年。在迷糊老师、菲菲公主的帮助下，猪猪侠接受了挑战，开始了这段充满了关心和情谊的旅程。    更多精彩震撼感觉，立即下载该款游戏尽情体验吧。玩家交流才是王道，讯易游戏玩家交流中心 QQ群：6306852-----------------生活要有激情，游戏要玩多彩(多彩游戏)。Colourfulgame (多彩游戏)，让你看看快乐游戏的颜色！精品推荐：1：《钟馗传》大战无头关羽，悲壮的剧情伴随各朝英灵反攻地府！2：《中华群英》将和赵云，项羽，岳飞等猛将作战，穿越各朝代抗击日寇。良品推荐：1：《赌王争霸之斗地主》易飞会在四角恋中会选择谁？是否最终成赌神呢？2：勇者后裔和魔王紧缠一起，前代恩怨《圣火伏魔录》将为您揭示一切。  3：颠覆传统概念，恶搞+非主流？！誓必弄死搞残为止《爆笑飞行棋》。4：《中国象棋残局大师》快棋和人机模式让畅快对弈！一切“多彩游戏”资讯，点击Colourfulgame官网http://www.colourfulgame.com一切“多彩游戏”感言，交流Colourfulgame论坛http://121.33.203.124/forum/【客服邮箱】：xunyiwangluo@126.com">xunyiwangluo@126.com">xunyiwangluo@126.com【客服热线】：020-87588437']
                 
    for i in test_demo:
        print(i,":",model.predict(i))

《时光重返四十二难》恶搞唐增取经一款时下最热门的动画人物：猪猪侠，加上创新的故事背景，震撼的操作快感，成就了这部恶搞新作，现正恶搞上市，玩家们抢先赶快体验快感吧。游戏简介：被时光隧道传送到208年的猪猪侠，必须经历六七四十二难的考验，才能借助柯伊诺尔大钻石的力量，开启时光隧道，重返2008年。在迷糊老师、菲菲公主的帮助下，猪猪侠接受了挑战，开始了这段充满了关心和情谊的旅程。    更多精彩震撼感觉，立即下载该款游戏尽情体验吧。玩家交流才是王道，讯易游戏玩家交流中心 QQ群：6306852-----------------生活要有激情，游戏要玩多彩(多彩游戏)。Colourfulgame (多彩游戏)，让你看看快乐游戏的颜色！精品推荐：1：《钟馗传》大战无头关羽，悲壮的剧情伴随各朝英灵反攻地府！2：《中华群英》将和赵云，项羽，岳飞等猛将作战，穿越各朝代抗击日寇。良品推荐：1：《赌王争霸之斗地主》易飞会在四角恋中会选择谁？是否最终成赌神呢？2：勇者后裔和魔王紧缠一起，前代恩怨《圣火伏魔录》将为您揭示一切。  3：颠覆传统概念，恶搞+非主流？！誓必弄死搞残为止《爆笑飞行棋》。4：《中国象棋残局大师》快棋和人机模式让畅快对弈！一切“多彩游戏”资讯，点击Colourfulgame官网http://www.colourfulgame.com一切“多彩游戏”感言，交流Colourfulgame论坛http://121.33.203.124/forum/【客服邮箱】：xunyiwangluo@126.com">xunyiwangluo@126.com">xunyiwangluo@126.com【客服热线】：020-87588437 : 游戏


In [4]:
!pip install keras

    100% |████████████████████████████████| 378kB 98.4MB/s eta 0:00:01eta 0:00:01
You are using pip version 9.0.1, however version 20.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
